<h1 style="font-family:Impact,Arial;font-size:30px;">37004 Interest Rates and Credit Risk Models - Spring 2024</h1>
<h1 style="font-family:Impact,Arial;font-size:45px;">Assignment Part 2</h1>
<h2 style="font-family:Arial;">Erik Schl&ouml;gl</h2>
<p><small> School of Mathematical &amp; Physical Sciences<br>
University of Technology Sydney
</small></p>
<p>
<a href="mailto:Erik.Schlogl@uts.edu.au?Subject=37000 JIT" target="_blank">
<small><font color=MediumVioletRed>Erik.Schlogl@uts.edu.au</font></small></a>
</p>
<hr style="height:5px;border:none;color:#333;background-color:#333;" />

Consider the data files <code>daily-treasury-rates2018.csv</code>, <code>daily-treasury-rates2019.csv</code>, <code>daily-treasury-rates2020.csv</code>, <code>daily-treasury-rates2021.csv</code> and <code>daily-treasury-rates2022.csv</code>. They contain the United States Treasury's "Daily Yield Curve" data (sourced from www.treasury.gov) for the years 2018 to 2022. These rates are what the US Treasury calls "CMT rates" (see
https://www.treasury.gov/resource-center/faqs/Interest-Rates/Pages/faq.aspx).

<H2>Task 1:</H2>
Convert these rates into continuously compounded yields. <I>(1 mark)</I>

In [8]:
# Python 3.12.5
import pandas as pd # 2.2.2
import numpy as np # 1.26.4
import datetime as dt
from bisect import bisect_left as bl
from scipy.optimize import fsolve # 1.14.1
from scipy.stats import norm
from numba import jit, njit # 0.60.0
from math import fabs, erf, erfc

NPY_SQRT1_2 = 1.0/ np.sqrt(2)

tr18 = pd.read_csv("daily-treasury-rates2018.csv")
tr19 = pd.read_csv("daily-treasury-rates2019.csv")
tr20 = pd.read_csv("daily-treasury-rates2020.csv")
tr21 = pd.read_csv("daily-treasury-rates2021.csv")
tr22 = pd.read_csv("daily-treasury-rates2022.csv")

data = pd.concat([tr18, tr19, tr20, tr21, tr22], ignore_index=True)

# Renaiming columns with an appropriate convention
data = data.rename(
    columns={
        "1 Mo": 1 / 12,
        "2 Mo": 1 / 6,
        "3 Mo": 1 / 4,
        "6 Mo": 1 / 2,
        "1 Yr": 1,
        "2 Yr": 2,
        "3 Yr": 3,
        "5 Yr": 5,
        "7 Yr": 7,
        "10 Yr": 10,
        "20 Yr": 20,
        "30 Yr": 30,
    }
)
data["Date"] = pd.to_datetime(data["Date"])
data.set_index("Date", inplace=True)
data.sort_values(by="Date", inplace=True)

# Converting the simply compounded rates to APY using the formula provided by the treasury.gov Website
data = (1 + data / 200) ** 2 - 1

for i in data.columns:
    data[i] = data[i].apply(lambda x: -np.log((1 + x) ** (-i)) / i)
data

,0.083333,0.166667,0.25,0.5,1,2,3,5,7,10,20,30
Date,,,,,,,,,,,,
2018-01-02,0.012859,NaN,0.014348,0.016036,0.018217,0.019108,0.020000,0.022374,0.023660,0.024450,0.026227,0.027904
2018-01-03,0.012859,NaN,0.014051,0.015837,0.018019,0.019307,0.020099,0.022374,0.023561,0.024252,0.026030,0.027609
2018-01-04,0.012759,NaN,0.014051,0.015936,0.018118,0.019505,0.020396,0.022572,0.023660,0.024450,0.026030,0.027707
2018-01-05,0.012660,NaN,0.013852,0.015738,0.017919,0.019505,0.020495,0.022770,0.023857,0.024549,0.026227,0.027904
2018-01-08,0.012958,NaN,0.014448,0.015936,0.017820,0.019505,0.020594,0.022770,0.023956,0.024746,0.026326,0.027904
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-29,0.024351,0.027313,0.029482,0.032927,0.034009,0.033911,0.034206,0.032436,0.031845,0.030959,0.034697,0.032239
2022-08-30,0.024154,0.027115,0.029482,0.032829,0.034501,0.034304,0.034402,0.032436,0.031944,0.030861,0.034599,0.032042
2022-08-31,0.023857,0.027017,0.029383,0.032927,0.034697,0.034206,0.034304,0.032731,0.032239,0.031255,0.034992,0.032436


In this data set, the <I>time to maturity</I> (not the time of maturity) is constant in each column. Thus, from the data in each column, we can obtain an estimate of the variance of the day-to-day changes in a continuously compounded yield with a given time to maturity. In order to relate this back to an HJM model, we need the dynamics of rates with fixed time to maturity in the model. 

We start with the instantaneous forward rates. The instantaneous forward rates $f(t,T)$ modelled in HJM have a fixed time of maturity $T$, with dynamics
$$
f(t,T)=f(0,T)+\int_0^t\sigma(u,T)\int_u^T\sigma(u,s)dsdu+\int_0^t\sigma(u,T)dW_{\beta}(u)\quad\forall\
t\in[0,T]
$$
Define instantaneous forward rates $r(t,x)$ with constant time to maturity $x$ by
$$
r(t,x)=f(t,t+x)
$$
The dynamics of $r(t,x)$ are given by the <I>Musiela equation</I> (for a proof, see e.g. Section 18.3 of Bj&ouml;rk, T. (1998) <I>Arbitrage Theory in Continuous Time</I>, Oxford University Press)
$$
r(t,x)=r(0,x)+\int_0^t\left(\frac{\partial}{\partial x}r(u,x)+\sigma(u,u+x)\int_0^x\sigma(u,u+s)ds\right)du+\int_0^t\sigma(u,u+x)dW_{\beta}(u)
$$

The continously compounded yield with constant time to maturity $x$ is
$$
y(t,x)=\frac1x\int_0^xr(t,s)ds
$$
Thus we have for the change in the continously compounded yield with constant time to maturity over a $\Delta t$ time step:
$$
y(t+\Delta t,x)-y(t,x)=\frac1x\int_0^x\left(\int_t^{t+\Delta t}\left(\frac{\partial}{\partial s}r(u,s)+\sigma(u,u+s)\int_0^s\sigma(u,u+v)dv\right)du+\int_t^{t+\Delta t}\sigma(u,u+s)dW_{\beta}(u)\right)ds
$$
Note that the drift will contribute to the variance of changes in the continously compounded yield via
$$
\frac{\partial}{\partial s}r(u,s),
$$
but for practical purposes this is negligible. Thus
\begin{align}
\text{Var}[y(t+\Delta t,x)-y(t,x)] &\approx& \text{Var}\left[\frac1x\int_0^x\int_t^{t+\Delta t}\sigma(u,u+s)dW_{\beta}(u)ds\right]\\
&=& \text{Var}\left[\frac1x\int_t^{t+\Delta t}\int_0^x\nu e^{-a(u+s-u)}dsdW_{\beta}(u)\right]\\
&=& \text{Var}\left[\frac1x\int_t^{t+\Delta t}\frac{\nu}a(1-e^{-ax})dW_{\beta}(u)\right]\\
&=& \left(\frac1x\frac{\nu}a(1-e^{-ax})\right)^2\Delta t
\end{align}

## Task 2:
Assuming a one-factor Gauss/Markov HJM model with mean reversion fixed at $a = 0.1$, use all the data for the six-month yields to determine an appropriate choice of the volatility level $\nu$ (notation as above). For simplicity, you may assume that the data observations are equally spaced in calendar time. *(3 marks)*

Extracting $\nu$ from $(4)$ we get $$\nu = \frac{xa\sqrt{\frac{\text{Var}[y(t+\Delta t,x)-y(t,x)]}{\Delta t}}}{1-e^{-ax}}$$

In [9]:
data_6m = data[0.5].diff().iloc[1:]
var = np.var(data_6m)

# Setting a fixed time length (+(2/3) is here to make sure we get 4.67 years since we only have the data of 8 months in 2022)
Delta_t = (data_6m.index[-1].year - data_6m.index[0].year + (2 / 3)) / len(data_6m)
x = data_6m.name
a = 0.1

nu = (x * a * np.sqrt(var / Delta_t)) / (1 - np.exp(-a * x))

print("Nu : {} %".format(nu * 100))

Nu : 0.48522823972051443 %


<H2>Task 3:</H2>

Assuming a one-factor Gauss/Markov HJM model, use all the data for the one-month and thirty-year yields to determine an appropriate choice of the mean reversion parameter $a$ and volatility level $\nu$. For simplicity, you may assume that the data observations are equally spaced in calendar time. <I>(3 marks)

</I>

In [10]:
import scipy as sp

def f(a_nu_vect, data, Delta_t):

    var_1m = np.std(data[1 / 12].diff().iloc[1:]) ** 2
    var_30Y = np.std(data[30].diff().iloc[1:]) ** 2
    # This is our first equation using the data from the 1m bond
    res1 = ((1 / 12) * a_nu_vect[0] * np.sqrt(var_1m / Delta_t)) / (
        1 - np.exp(-a_nu_vect[0] * (1 / 12))
    ) - a_nu_vect[1]

    # This is our second equation using the data from the 30Y bond
    res2 = (30 * a_nu_vect[0] * np.sqrt(var_30Y / Delta_t)) / (
        1 - np.exp(-a_nu_vect[0] * 30)
    ) - a_nu_vect[1]

    return (res1, res2)

a, nu = sp.optimize.fsolve(f, [0.01, 0.01], (data, Delta_t))

A way in which the performance of a model for risk management purposes can be <B>backtested</B> is to use historical interest data to analyse what would have happened if one had used the model to hedge a derivative security in the past. This motivates the following Task:

<H2>Task 4:</H2>
Suppose that at the beginning of January 2022 you have sold an at-the-money European receiver swaption, expiring in six months, where the frequency of payments of the underlying swap is quarterly and the swap ends five years after the swaption expiry. If you hedge this swaption with rebalancing on every day for which there is data, using the natural hedge instruments of the constituent zero coupon bond options, what would be your profit/loss when this option matures? Assume that swaption price and hedges are calculated using a one-factor Gauss/Markov HJM model with the parameters determined as in Task 3, but <B>using only data which would have been available at the time the swaption was sold</B>. Assume that any profit/loss is invested/borrowed by buying/selling the zero coupon bond maturing at the same time as the swaption. Use loglinear interpolation of zero coupon bond prices, but only where necessary. Repeat
this for a (otherwise identical) swaption sold at the beginning of, respectively, April 2021, July 2021, and October 2021. Discuss the possible causes of profit/loss. <I>(13 marks)</I>

The first thing we need is to compute the swap yield of the receiver swaption. A receiver swaption can be seen as an option to enter in a swap with swap yield $\ell$ at a future date $T_0$.

From slide 82 of lecture 2-3, we get that the value at time $0$ of a forward start swap contract from the viewpoint of the side paying fixed on a tenor $\mathbb{T} = \{T_0,...,T_N\} $ is given by :

$$S(t, \mathbb{T}, \ell_k)=B(0,T_0)-B(0,T_N)-\ell \sum_{i=1}^{N} (T_i - T_{i-1}) B(0, T_i)$$

Knowing the swap was sold at the money, we have $S(t, \mathbb{T}, \ell_k) = 0$ making us solve for $\ell$ :

$$B(0,T_0)-B(0,T_N)-\ell \sum_{i=1}^{N} (T_i - T_{i-1}) B(0, T_i) = 0$$

$$\ell =\frac{B(0,T_0)-B(0,T_N)}{\sum_{i=1}^{N} (T_i - T_{i-1}) B(0, T_i)}  $$

$$-\int_{t}^{T}\int_{0}^{t} \frac{v^2}{a} e^{-a(u-s)} \left(1 - e^{-a(u-s)}\right) \, ds \, du$$

$$=\frac{v^2}{4a^3}[(e^{-2a(T-t)} - e^{-2aT}) - (1 - e^{-2at})] - \frac{v^2}{a^3}[(e^{-a(T-t)} - e^{-aT}) - (1 - e^{-at})]$$

<!-- $$\mathit{payoff}_{T_0} = [\ell_k \sum_{i=1}^{N} (T_i - T_{i-1}) B(T_0, T_i) - (1 - B(T_0, T_N))]_+ $$
$$ = [B(T_0, T_N) + \ell_k \sum_{i=1}^{N} (T_i - T_{i-1}) B(T_0, T_i) - 1]_+ $$

We see that $B(T_0, T_N) + \ell_k \sum_{i=1}^{N} (T_i - T_{i-1}) B(T_0, T_i)$ can be interpreted as the value at $T_0$ of a coupon bond with rate $\ell_k$

Therefore, taking $\mathbb{V}(T_0, T_N, \ell_k) = B(T_0, T_N) + \ell_k \sum_{i=1}^{N} (T_i - T_{i-1}) B(T_0, T_i)$, the price of the swaption can be seen as a call option on a coupon bond with rate $\ell_k$ and strike 1 as the following equation suggests :

$$\mathbb{C}(0, T_0, T_N, \ell_k) = B(0, T_0)\mathbb{E}_{T_0}[ (\mathbb{V}(T_0, T_N, \ell_k) - 1)\mathbb{I}_{\{\mathbb{V} (T_0, T_N, \ell_k) > 1\}}]$$

$$B(T_0, T_N) + \ell_k \sum_{i=1}^{N} (T_i - T_{i-1}) B(T_0, T_i) - 1 = 0$$ -->

In [11]:
data_copy = data.copy()
data_copy.columns = (data_copy.columns * 365).astype(np.int64)

In [12]:
# Tutorial 1 Question 3
def swaption_hedge(data, a, nu, date, swaption_maturity, swap_maturity, freq):

    def ZCB(data, date, maturity):

        temp_data = data.loc[date.strftime("%Y-%m-%d")]

        if maturity == 0:
            return 1
        elif maturity > 0 and maturity <= 10950:
            if maturity in data.columns:
                r = temp_data[maturity]
                return np.exp(-(maturity / 365) * r)
            else:
                index_ = bl(data.columns, maturity)

                maturity_left = data.columns[index_ - 1]
                maturity_right = data.columns[index_]
                
                r_left = temp_data[maturity_left]
                r_right = temp_data[maturity_right]

                if index_ == 0:
                    maturity_left = 0

                ZCB_left = np.exp(-(maturity_left / 365) * r_left)
                ZCB_right = np.exp(-(maturity_right / 365) * r_right)

                return np.exp(
                    np.log(ZCB_left)
                    + (
                        ((maturity - maturity_left) / (maturity_right - maturity_left))
                        * np.log(ZCB_right / ZCB_left)
                    )
                )
        else:
            raise ValueError("Time gap must be more than 0 and less or equal to 30")

    def find_swap_yield(data, date, swaption_maturity, swap_maturity, freq):

        time_gaps = [1 / freq] * (freq * swap_maturity)
        cum_sum_time_gaps = [((1 + i) / freq) + swaption_maturity for i in range(len(time_gaps))]

        ZCBs = [ZCB(data, date, int((swaption_maturity + i) * 365)) for i in cum_sum_time_gaps]

        B_0_T0 = ZCB(data, date, int(swaption_maturity * 365))
        B_0_TN = ZCB(data, date, int((swaption_maturity + swap_maturity) * 365))

        numerator = B_0_T0 - B_0_TN
        denominator = np.dot(time_gaps, ZCBs)

        return numerator / denominator

    def ZCB_r(data, date, nu, a, t, T, r):

        if (t > 0) and (T > 0) and (t <= 30) and (T <= 30) and (t < T):
            
            def A(nu, a, t, T):
                int1 = np.exp(-2 * a * (T - t)) - np.exp(- 2 * a * T)
                int2 = 1 - np.exp(- 2 * a * t)
                int3 = np.exp(-a * (T - t)) - np.exp(- a * T)
                int4 = 1 - np.exp(- a * t)

                c1 = (nu**2) / (4 * a**3)
                c2 = (nu**2) / (a**3)

                temp = (c1 * (int1 - int2)) - (c2 * (int3 - int4))

                return (ZCB(data, date, int(T * 365)) / ZCB(data, date, int(t * 365))) * np.exp(temp)

            def B(a, t, T):
                
                return (1 / a) * (np.exp(- a * t) - np.exp(- a * T))
            
            return A(nu, a, t, T) * np.exp(- B(a, t, T) * r)
        
        elif t == T:
            return 1
        elif t > T:
            raise ValueError("t must be less than T")
        else:
            raise ValueError("Time gaps must be more than 0 and less or equal to 30")
    
    def find_r(data, swap_rate, date,a, nu, swaption_maturity, swap_maturity, freq):

        def to_solve(r, data, swap_rate, date, a, nu, swaption_maturity, swap_maturity, freq):

            r = r[0]
            times = [1 / freq] * (freq * swap_maturity)
            ZCBs = [ZCB_r(data, date, nu, a, swaption_maturity, swaption_maturity + i, r) for i in list(np.cumsum(times))]
            BT0_TN = ZCB_r(data, date, nu, a, swaption_maturity, swaption_maturity + np.cumsum(times)[-1], r)

            return BT0_TN - 1 + np.dot(ZCBs, times) * swap_rate
        
        return fsolve(to_solve, [0], (data, swap_rate, date, a, nu, swaption_maturity, swap_maturity, freq))[0]

    def B_S_std(T0, T1, a, nu):
        c1 = (- (nu ** 2)) / (2 * (a ** 3))
        c2 = ((- (nu ** 2)) / (a ** 3)) * np.exp(- a * (T1 + T0))
        
        int1 = 1 - np.exp(- 2 * a * T0)
        int2 = np.exp(- 2 * a * (T1 - T0)) - np.exp(- 2 * a * T1)

        return (c1 * int1) + (c1 * int2) - (c2 * int1)
    
    def h1(data, date, K, T0, T1, a, nu):

        X = np.log(ZCB(data, date, T1) / (ZCB(data, date, T0) * K))
        mean = .5 * B_S_std(T0, T1, a, nu)
        stdev = np.sqrt(B_S_std(T0, T1, a, nu))

        return (X + mean) / stdev
    
    def h2(data, date, K, T0, T1, a, nu):
        X = np.log(ZCB(data, date, T1) / (ZCB(data, date, T0) * K))
        mean = .5 * B_S_std(T0, T1, a, nu)
        stdev = np.sqrt(B_S_std(T0, T1, a, nu))

        return (X - mean) / stdev
    
    def B_S_call_price(data, date, K, T0, T1, a, nu):

        h1_ = h1(data, date, K, T0, T1, a, nu)
        h2_ = h2(data, date, K, T0, T1, a, nu)

        return (ZCB(data, date, T1) * norm.cdf(h1_)) - (ZCB(data, date, T0) * norm.cdf(h2_))

    def B_S_delta(data, date, K, T0, T1, a, nu):
        return norm(h1(data, date, K, T0, T1, a, nu))

    swap_rate = find_swap_yield(data, date, swaption_maturity, swap_maturity, 4)
    r = find_r(data, swap_rate, date, a, nu, swaption_maturity, swap_maturity, freq)
    print(ZCB(data, date, 30*365), swap_rate, r)
    K = ZCB_r(data, date, nu, a, swaption_maturity, swap_maturity, r)
    # print(swap_rate, B_S_call_price(data, date, K, swaption_maturity, swap_maturity, a, nu))

    

dates = [
    dt.date(2022, 1, 3),
    dt.date(2021, 4, 1),
    dt.date(2021, 7, 1),
    dt.date(2021, 10, 1),
]
for date in dates:
    swaption_hedge(data_copy, a, nu, date, 0.5, 5, 4)

0.5488170832411898 0.01548915486829566 0.00015658050947977382
0.49761676132858357 0.01143394952959279 0.00010922163423811533
0.5391247279425175 0.010921681986738572 0.00010164720159227221
0.5439489583400844 0.01129127040632377 0.00010555450986273993


In [13]:
rates = [np.float64(i) for i in list(data.loc[date.strftime("%Y-%m-%d")].values)]
maturities = [np.float64(i) for i in list(data.columns)]


@njit(cache=True)
def find_index(array, value):
    if value in array:
        for i in range(len(array)):
            if value == array[i]:
                return i
    raise ValueError('Impossible to find "value" in "array"')
find_index([0, 1], 0)


@njit(cache=True)
def ZCB(rates, maturities, maturity):

    if maturity == 0:
        return 1
    elif maturity > 0 and maturity <= 30:
        if maturity in maturities:
            r = rates[find_index(maturities, maturity)]
            return np.exp(- maturity * r)
        else:

            maturity_left = 0
            maturity_right = 0

            temp_maturities = [0] + maturities

            for i in range(len(temp_maturities) - 1):
                if temp_maturities[i] < maturity and maturity < temp_maturities[i + 1]:
                    maturity_left = temp_maturities[i]
                    maturity_right = temp_maturities[i + 1]
                    break
            r_right = rates[find_index(maturities, maturity_right)]
            r_left = 0

            if maturity_left != 0:
                maturity_left = rates[find_index(maturities, maturity_left)]

            ZCB_left = np.exp(-maturity_left * r_left)
            ZCB_right = np.exp(-maturity_right * r_right)

            return np.exp(
                np.log(ZCB_left)
                + (
                    ((maturity - maturity_left) / (maturity_right - maturity_left))
                    * np.log(ZCB_right / ZCB_left)
                )
            )
    else:
        raise ValueError("Time gap must be more than 0 and less or equal to 30")
ZCB(rates, maturities, 1)


@njit(cache=True)
def find_swap_yield(rates, maturities, swaption_maturity, swap_maturity, freq):

    time_gaps = [1 / freq] * (freq * swap_maturity)
    cum_sum_time_gaps = [((1 + i) / freq) for i in range(len(time_gaps))]
    ZCBs = [ZCB(rates, maturities, swaption_maturity + i) for i in cum_sum_time_gaps]

    B_0_T0 = ZCB(rates, maturities, swaption_maturity)

    numerator = B_0_T0 - ZCBs[-1]
    denominator = np.sum(np.array(ZCBs).astype(np.float64)) / 4

    return numerator / denominator
find_swap_yield(rates, maturities, 0.5, 5, 4)


@njit(cache=True)
def ZCB_r(rates, maturities, nu, a, t, T, r):

    if (t > 0) and (T > 0) and (t <= 30) and (T <= 30) and (t < T):
        
        int1 = np.exp(-2 * a * (T - t)) - np.exp(- 2 * a * T)
        int2 = 1 - np.exp(- 2 * a * t)
        int3 = np.exp(-a * (T - t)) - np.exp(- a * T)
        int4 = 1 - np.exp(- a * t)

        c1 = (nu**2) / (4 * a**3)
        c2 = (nu**2) / (a**3)

        temp = (c1 * (int1 - int2)) - (c2 * (int3 - int4))

        A =  (ZCB(rates, maturities, T) / ZCB(rates, maturities, t)) * np.exp(temp)
        B = (1 / a) * (np.exp(- a * t) - np.exp(- a * T))
        
        return A * np.exp(- B * r)
    
    elif t == T:
        return 1
    
    elif t > T:
        raise ValueError("t must be less than T")
    
    else:
        raise ValueError("Time gaps must be more than 0 and less or equal to 30")
ZCB_r(rates, maturities, nu, a, 0.1, 30, 0.01)


@njit(cache=True)
def to_solve_numba(r, rates, maturities, swap_rate, a, nu, swaption_maturity, swap_maturity, freq):
    
    time_gaps = [1 / freq] * (freq * swap_maturity)
    cum_sum_time_gaps = [((1 + i) / freq) for i in range(len(time_gaps))]
    ZCBs = [ZCB_r(rates, maturities, nu, a, swaption_maturity, swaption_maturity + i, r) for i in cum_sum_time_gaps]

    result = ZCBs[-1] - 1 + (np.sum(np.array(ZCBs).astype(np.float64)) * (swap_rate / 4))

    return result
to_solve_numba(0.02, rates, maturities, 0.016690507191985667, nu, a, 0.5, 5, 4)


def to_solve(r, rates, maturities, swap_rate, a, nu, swaption_maturity, swap_maturity, freq):
    if type(r) != float:
        r = float(r[0])
    return to_solve_numba(r, rates, maturities, swap_rate, a, nu, swaption_maturity, swap_maturity, freq)


def find_r(rates, maturities, swap_rate, a, nu, swaption_maturity, swap_maturity, freq):
    return fsolve(to_solve, 0, (rates, maturities, swap_rate, a, nu, swaption_maturity, swap_maturity, freq))[0]


@njit(cache=True, fastmath=True)
def N(a):
    # https://github.com/cuemacro/teaching/blob/master/pythoncourse/notebooks/numba_example.ipynb

    if (np.isnan(a)):
        return np.nan

    x = a * NPY_SQRT1_2;
    z = fabs(x)

    if (z < NPY_SQRT1_2):
        y = 0.5 + 0.5 * erf(x)

    else:
        y = 0.5 * erfc(z)

        if (x > 0):
            y = 1.0 - y

    return y
N(0.4)


@njit(cache=True)
def B_S_var(T0, T1, a, nu):
    c1 = (nu ** 2) / (2 * (a ** 3))
    c2 = (-(nu ** 2)) / (a ** 3)
    
    int1 = np.exp(- 2 * a * (T1 - T0)) - np.exp(- 2 * a * T1)
    int2 = 1 - np.exp(- 2 * a * T0)
    int3 = np.exp(- a * (T1 - T0)) - np.exp(- a * (T1 + T0))

    return (c1 * int1) + (c1 * int2) + (c2 * int3)
B_S_var(0.1, 1, a, nu)


@njit(cache=True)
def h1(rates, maturities, K, T0, T1, a, nu):

    X = np.log(ZCB(rates, maturities, T1) / (ZCB(rates, maturities, T0) * K))
    mean = .5 * B_S_var(T0, T1, a, nu)
    stdev = np.sqrt(B_S_var(T0, T1, a, nu))

    return (X + mean) / stdev
h1(rates, maturities, 1, 0.1, 1, a, nu)


@njit(cache=True)
def h2(rates, maturities, K, T0, T1, a, nu):

    X = np.log(ZCB(rates, maturities, T1) / (ZCB(rates, maturities, T0) * K))
    mean = .5 * B_S_var(T0, T1, a, nu)
    stdev = np.sqrt(B_S_var(T0, T1, a, nu))

    return (X - mean) / stdev
h2(rates, maturities, 1, 0.1, 1, a, nu)


@njit(cache=True)
def B_S_price(rates, maturities, K, T0, T1, a, nu):

    h1_ = h1(rates, maturities, K, T0, T1, a, nu)
    h2_ = h2(rates, maturities, K, T0, T1, a, nu)

    return (ZCB(rates, maturities, T1) * N(h1_)) - (ZCB(rates, maturities, T0) * K * N(h2_))
B_S_price(rates, maturities, 1, 1, 2, a, nu)


@njit(cache=True)
def swaption_value(rates, maturities, r, swaption_maturity, swap_maturity, a, nu, swap_rate, freq):

    time_gaps = [1 / freq] * (freq * swap_maturity)
    cum_sum_time_gaps = [((1 + i) / freq) for i in range(len(time_gaps))]

    strikes = [ZCB_r(rates, maturities, nu, a, swaption_maturity, swaption_maturity + cum_sum_time_gaps[i], r) for i in range(len(cum_sum_time_gaps))]
    options = [B_S_price(rates, maturities, strikes[i], swaption_maturity, swaption_maturity + cum_sum_time_gaps[i], a, nu) for i in range(len(cum_sum_time_gaps))]

    return options[-1] + ((swap_rate / 4) * np.sum(np.array(options)))
swaption_value(rates, maturities, 0.02, 0.5, 5, a, nu, 0.02, 4)


@njit(cache=True)
def delta_hedge_weights(rates, maturities, r, swaption_maturity, swap_maturity, a, nu, swap_rate, freq):
    time_gaps = [1 / freq] * (freq * swap_maturity)
    cum_sum_time_gaps = [((1 + i) / freq) for i in range(len(time_gaps))]

    strikes = [ZCB_r(rates, maturities, nu, a, swaption_maturity, swaption_maturity + cum_sum_time_gaps[i], r) for i in range(len(cum_sum_time_gaps))]

    weights1 = [(swap_rate / freq) * strikes[i] * N(h2(rates, maturities, strikes[i], swaption_maturity, swaption_maturity + cum_sum_time_gaps[i], a, nu)) for i in range(len(strikes))]
    weights1[-1] = weights1[-1] + weights1[-1] / (swap_rate / freq)

    weights2 = [(swap_rate / freq) * N(h1(rates, maturities, strikes[i], swaption_maturity, swaption_maturity + cum_sum_time_gaps[i], a, nu)) for i in range(len(weights1))]
    weights2[-1] = weights2[-1] + weights2[-1] / (swap_rate / freq)
    
    return [np.sum(np.array(weights1))] + weights2
delta_hedge_weights(rates, maturities, 0.0001, 0.5, 5, a, nu, 0.02, 4)
pass

In [14]:
def swaption_hedge(data, date, swaption_maturity, swap_maturity, freq):

    new_data = data.loc[date.strftime("%Y-%m-%d") : (date + dt.timedelta(swaption_maturity * 365))].copy()
    a, nu = sp.optimize.fsolve(f, [0.01, 0.01], (new_data, Delta_t))

    rates = [float(i) for i in list(data.loc[date.strftime("%Y-%m-%d")].values)]
    maturities = [float(i) for i in list(data.columns)]
    swap_rate = find_swap_yield(rates, maturities, 0.5, 5, 4)
    r = find_r(rates, maturities, swap_rate, a, nu, swaption_maturity, swap_maturity, freq)
    print(
        swaption_value(rates, maturities, r, swaption_maturity, swap_maturity, a, nu, swap_rate, freq)

    )
    print()
    for today in new_data.index.values[1:]:
        data_today = data.loc[today]

    return 0

dates = [
    dt.date(2022, 1, 3),
    dt.date(2021, 4, 1),
    dt.date(2021, 7, 1),
    dt.date(2021, 10, 1),
]
for date in dates:
    print(date.strftime("%d %B %Y"), 'PnL :', swaption_hedge(data, date, 0.5, 5, 4))

0.009728704902972021

03 January 2022 PnL : 0
0.002215284894196747

01 April 2021 PnL : 0
0.004087079569286657

01 July 2021 PnL : 0
0.005267000997374193

01 October 2021 PnL : 0
